# List of topics for the final project



## Project 5 algorithmic fairness
Algorithm fairness is becoming a fundamental topic in ML. It is a complex ethical task to define what fairness is/means. Once we have defined quantitatively what fairness is then, from a mathematical perspective, the problem of  algorithmic fairness is very clear: it is a multi-objective optimization problem. There are multi objectives that we aim to optimize during data fitting, e.g., accuracy and fairness.

The goal of this project is to implement from scratch the "fair" **linear and nonlinear SVM** described in the following paper (see in particular Appendix A that reports the optimization problem)  

["Fairness Constraints: Mechanisms for Fair Classification"](https://arxiv.org/pdf/1507.05259.pdf)

and reproduce the experiments reported in the paper. In particular, apply your method to the Adult and Bank 
data sets.

Your notebook must include:
* a description (summary) of the algorithm presented in the above paper (focusing on SVM), similar to the theoretical details of logistic regression I wrote at the beginning of the notebook for e-tivity Task A (week 1&2). The reader must understand from your explanation the difference between standard SVM and the "fair" SVM.
* You implementation of the "fair" **linear and nonlinear SVM** using CVXOPT to solve data fitting (as I have shown in Week 3 webinar, see also example below). You should implement it as a Python class (similar to logistic regression class for E-tivity 1).
* A test of the input-output behavior of your algorithm. More clearly, you have to replicate the experiment results you find in Section 4.1 for Synthetic Data and Section 4.2 of the above paper for the Adult and Bank data sets.


Resources:
* Week 3 webinar slides with the details of the SVM algorithm;
* [Example](https://xavierbourretsicotte.github.io/SVM_implementation.html) about how to use the library CVXOPT to implement data fitting for standard SVM
* [fairness-in-machine-learning](https://towardsdatascience.com/a-tutorial-on-fairness-in-machine-learning-3ff8ba1040cb)
* (Optional) Multi-objective optimization and Pareto optimality see Book chapter 12 (of our Module's book).

**How to approach the problem** (this is just a suggestion).

You can start implementing linear SVM and apply it to the Synthetic Data experiment in Section 4.1 so that you can plot the classification line for standard linear SVM versus fair linear SVM.


In [1]:
import numpy as np
from sklearn import datasets
import pandas as pd 
from sklearn import preprocessing

# Import helper functions
from mlfromscratch.utils import train_test_split, normalize, accuracy_score, Plot
#from mlfromscratch.utils.kernels import *
from mlfromscratch.supervised_learning import SupportVectorMachine

from sklearn import svm

In [2]:
from __future__ import division, print_function
import numpy as np
import cvxopt
from cvxopt import matrix
from mlfromscratch.utils import train_test_split, normalize, accuracy_score
from mlfromscratch.utils.kernels import *
from mlfromscratch.utils import Plot

# Hide cvxopt output
cvxopt.solvers.options['show_progress'] = True

class SupportVectorMachine(object):
    """The Support Vector Machine classifier.
    Uses cvxopt to solve the quadratic optimization problem.

    Parameters:
    -----------
    C: float
        Penalty term.
    kernel: function
        Kernel function. Can be either polynomial, rbf or linear.
    power: int
        The degree of the polynomial kernel. Will be ignored by the other
        kernel functions.
    gamma: float
        Used in the rbf kernel function.
    coef: float
        Bias term used in the polynomial kernel function.
    """
    def __init__(self, C=None, kernel=rbf_kernel, sensible_feature=None, power=4, gamma=None, coef=4):
        self.C = C
        self.kernel = kernel
        self.power = power
        self.gamma = gamma
        self.coef = coef
        self.lagr_multipliers = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.intercept = None
        self.fairness = False if sensible_feature is None else True
        self.sensible_feature = sensible_feature        

    def fit(self, X, y):

        n_samples, n_features = np.shape(X)

        # Set gamma to 1/n_features by default
        if not self.gamma:
            self.gamma = 1 / n_features

        # Initialize kernel method with parameters
        self.kernel = self.kernel(
            power=self.power,
            gamma=self.gamma,
            coef=self.coef)

        # Calculate kernel matrix
        kernel_matrix = np.zeros((n_samples, n_samples))
        
        for i in range(n_samples):
            for j in range(n_samples):
                kernel_matrix[i, j] = self.kernel(X[i], X[j])
                
        if self.fairness:
            self.values_of_sensible_feature = list(set(self.sensible_feature))
            self.list_of_sensible_feature_train = self.sensible_feature
            self.val0 = np.min(self.values_of_sensible_feature)
            self.val1 = np.max(self.values_of_sensible_feature)
            print(f"val0={self.val0}, val1={self.val1}")
            self.set_A1 = [idx for idx, ex in enumerate(X) if y[idx] == 1
                           and self.sensible_feature[idx] == self.val1]
            self.set_not_A1 = [idx for idx, ex in enumerate(X) if y[idx] == 1
                               and self.sensible_feature[idx] == self.val0]
            self.set_1 = [idx for idx, ex in enumerate(X) if y[idx] == 1]
            self.n_A1 = len(self.set_A1)
            self.n_not_A1 = len(self.set_not_A1)
            self.n_1 = len(self.set_1)  
            
        # Define the quadratic optimization problem
        P = cvxopt.matrix(np.outer(y, y) * kernel_matrix, tc='d')
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1, n_samples), tc='d')
        b = cvxopt.matrix(0, tc='d')

        if not self.C:
            G = cvxopt.matrix(np.identity(n_samples) * -1)
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            G_max = np.identity(n_samples) * -1
            G_min = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((G_max, G_min)))
            h_max = cvxopt.matrix(np.zeros(n_samples))
            h_min = cvxopt.matrix(np.ones(n_samples) * self.C)
            h = cvxopt.matrix(np.vstack((h_max, h_min)))
            
        # Stack the fairness constraint
        if self.fairness:
            tau = [(np.sum(kernel_matrix[self.set_A1, idx]) / self.n_A1) - (np.sum(kernel_matrix[self.set_not_A1, idx]) / self.n_not_A1) for idx in range(len(y))]
            print(f"Tauji kee value {tau[0:10]}")
            fairness_line = matrix(y * tau, (1, n_samples), 'd')
            print("Fairness_line", fairness_line)
            A = cvxopt.matrix(np.vstack([A, fairness_line]))
            b = cvxopt.matrix([0.0, 0.0])            
            
        print("P.shape", P.size)
        print("q.shape", q.size)
        print("G.shape", G.size)
        print("h.shape", h.size)
        print("A.shape", A.size)
        print("b.shape", b.size)

        # Solve the quadratic optimization problem using cvxopt
        minimization = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        lagr_mult = np.ravel(minimization['x'])

        # Extract support vectors
        # Get indexes of non-zero lagr. multipiers
        idx = lagr_mult > 1e-7
        # Get the corresponding lagr. multipliers
        self.lagr_multipliers = lagr_mult[idx]
        # Get the samples that will act as support vectors
        self.support_vectors = X[idx]
        # Get the corresponding labels
        self.support_vector_labels = y[idx]

        # Calculate intercept with first support vector
        self.intercept = self.support_vector_labels[0]
        for i in range(len(self.lagr_multipliers)):
            self.intercept -= self.lagr_multipliers[i] * self.support_vector_labels[
                i] * self.kernel(self.support_vectors[i], self.support_vectors[0])

    def predict(self, X):
        y_pred = []
        # Iterate through list of samples and make predictions
        for sample in X:
            prediction = 0
            # Determine the label of the sample by the support vectors
            for i in range(len(self.lagr_multipliers)):
                prediction += self.lagr_multipliers[i] * self.support_vector_labels[
                    i] * self.kernel(self.support_vectors[i], sample)
            prediction += self.intercept
            y_pred.append(np.sign(prediction))
        return np.array(y_pred)

In [3]:
import os,sys
sys.path.insert(0, './fair-classification3/fair_classification') # the code for fair classification is in this directory

import urllib.request, urllib.error, urllib.parse
import numpy as np
from random import seed, shuffle
from sklearn import preprocessing
import pickle

SEED = 1122
seed(SEED) # set the random seed so that the random permutations can be reproduced again
np.random.seed(SEED)

In [19]:
"""
    The adult dataset can be obtained from: http://archive.ics.uci.edu/ml/datasets/Adult
    The code will look for the data files (adult.data, adult.test) in the present directory, if they are not found, it will download them from UCI archive.
"""

def check_data_file(fname):
    files = os.listdir(".") # get the current directory listing
    print("Looking for file '%s' in the current directory..." % fname)

    if fname not in files:
        print("'%s' not found! Downloading from UCI Archive..." % fname)
        addr = "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/%s" % fname
        response = urllib.request.urlopen(addr)
        data = response.read()
        fileOut = open(fname, "w")
        fileOut.write(data)
        fileOut.close()
        print("'%s' download and saved locally.." % fname)
    else:
        print("File found in current directory..")
    
    print()
    return

        
def load_bank_data(load_data_size=None, drop_sensitive=True):

   # adult data comes in two different files, one for training and one for testing, however, we will combine data from both the files
    attrs = ["age","job","marital","education","default","balance","housing","loan","contact","day","month","duration","campaign","pdays","previous","poutcome","y"] 
    
    data = pd.read_csv("dataset/bank/bank.csv", skipinitialspace=True, delimiter=';') 

    data['month-seasonal'] = data['month'].apply(lambda x: 'q1' if x in ["jan", "feb", "mar"] else x )
    data['month-seasonal'] = data['month-seasonal'].apply(lambda x: 'q2' if x in ["apr", "may", "jun"] else x )
    data['month-seasonal'] = data['month-seasonal'].apply(lambda x: 'q3' if x in ["jul", "aug", "sep"] else x )
    data['month-seasonal'] = data['month-seasonal'].apply(lambda x: 'q4' if x in ["oct", "nov", "dec"] else x )
    
    data['age-range'] = data['age'].apply(lambda x: 0.0 if (x >=25 and x<=60)  else 1.0 )

    data.drop(['month'], inplace=True, axis=1)
    
    encoded_object_df = pd.DataFrame()

    for column in ['job', 'marital', 'education', 'default', 'housing', 'loan','contact', 'poutcome','y','month-seasonal']:
        encoded_object_df = pd.concat([encoded_object_df,pd.get_dummies(data[column], prefix=column, drop_first=True)] ,axis=1)
    
    min_max_scaler = preprocessing.MinMaxScaler()

    cols_to_scale = ['balance', 'day','duration', 'campaign', 'pdays','previous']

    encoded_int_df = data[cols_to_scale]

    encoded_int_df[cols_to_scale] = min_max_scaler.fit_transform(encoded_int_df[cols_to_scale])
    
    final_df = pd.concat([encoded_object_df, encoded_int_df, data['age-range']], axis=1)
    
    final_df = final_df.sample(n=load_data_size, random_state=1234)
    #final_df = final_df.iloc[0:load_data_size]
    print(final_df.columns)
    print(final_df['y_yes'].value_counts())
    
    y = np.where(final_df['y_yes'] == 0, 0, 1)
    x_sensitive = np.array(final_df['age-range'])
    
    
    
    if drop_sensitive:
        final_df.drop(['age-range'], axis=1, inplace=True)
        
    final_df.drop(['y_yes'], axis=1, inplace=True)
        
    X = np.array(final_df)
    
    print(f"Shapes of X = {X.shape}, y={y.shape}, x_sensitive={x_sensitive.shape}")
    
    return X, y, x_sensitive

In [26]:
data_count=1000
X, y, x_control = load_bank_data(load_data_size=data_count)


Index(['job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'education_secondary',
       'education_tertiary', 'education_unknown', 'default_yes', 'housing_yes',
       'loan_yes', 'contact_telephone', 'contact_unknown', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown', 'y_yes', 'month-seasonal_q2',
       'month-seasonal_q3', 'month-seasonal_q4', 'balance', 'day', 'duration',
       'campaign', 'pdays', 'previous', 'age-range'],
      dtype='object')
0    879
1    121
Name: y_yes, dtype: int64
Shapes of X = (1000, 33), y=(1000,), x_sensitive=(1000,)


/home/vipulpopat/Tools/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/vipulpopat/Tools/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [27]:
print(X.shape)

(1000, 33)


In [28]:
print(y.shape)

(1000,)


In [29]:
def linear_kernel_cust(x1, x2):
    return np.dot(x1, x2)

def linear_kernel(**kwargs):
    def f(x1, x2):
        return np.dot(x1, x2)
    return f


def polynomial_kernel(power, coef, **kwargs):
    def f(x1, x2):
        return (np.dot(x1, x2) + coef)**power
    return f


def rbf_kernel(gamma, **kwargs):
    def f(x1, x2):
        distance = np.linalg.norm(x1 - x2) ** 2
        return np.exp(-gamma * distance)
    return f

def compute_p_rule(x_control, class_labels):

    print(f'x_controls={x_control}')
    print(f'class_labels={class_labels}')
    """ Compute the p-rule based on Doctrine of disparate impact """

    non_prot_all = sum(x_control == 1.0) # non-protected group
    prot_all = sum(x_control == 0.0) # protected group
    non_prot_pos = sum(class_labels[x_control == 1.0] == 1.0) # non_protected in positive class
    prot_pos = sum(class_labels[x_control == 0.0] == 1.0) # protected in positive class
    
    print(f'non_prot_all={non_prot_all}, prot_all={prot_all}, non_prot_pos={non_prot_pos}, prot_pos={prot_pos}')
    
    
    frac_non_prot_pos = float(non_prot_pos) / float(non_prot_all)
    frac_prot_pos = float(prot_pos) / float(prot_all)
    p_rule = (frac_prot_pos / frac_non_prot_pos) * 100.0
    print ()
    print(("Total data points: %d" % (len(x_control))))
    print(("# non-protected examples: %d" % (non_prot_all)))
    print(("# protected examples: %d" % (prot_all)))
    print(("Non-protected in positive class: %d (%0.0f%%)" % (non_prot_pos, non_prot_pos * 100.0 / non_prot_all)))
    print(("Protected in positive class: %d (%0.0f%%)" % (prot_pos, prot_pos * 100.0 / prot_all)))
    print(("P-rule is: %0.0f%%" % ( p_rule )))
    return p_rule



In [30]:
# base p_rule calcualtions on the original data
print("p_rule on base data = ", compute_p_rule(x_control, y))

x_controls=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 

In [34]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#X_control_train, X_control_test, y_train, y_test = train_test_split(x_control, y, test_size=0.33)

C_1=1
for C in [C_1]:
    
    for feature in [None, x_control]:
        p_feature = None
        if (feature is not None):
            p_feature = 'With Fairness Constraints'
        print(f'C={C},{p_feature}')
        clf = SupportVectorMachine(kernel=polynomial_kernel, sensible_feature=feature, C=C, power=4, coef=1)
        #clf = SupportVectorMachine(kernel=polynomial_kernel, sensible_feature=x_control)
        #clf.fit(X_train, y_train)
        clf.fit(X, y)
        #y_pred = clf.predict(X_test)
        y_pred = clf.predict(X)

        #accuracy = accuracy_score(y_test, y_pred)
        accuracy = accuracy_score(y, y_pred)
        print ("Accuracy:", accuracy)
        print(f"p_rule on = {p_feature} = {compute_p_rule(x_control, y_pred)}")
        print("=====================================================")
        # Reduce dimension to two using PCA and plot the results
        #Plot().plot_in_2d(X, y_pred, title="Support Vector Machine", accuracy=accuracy)

C=1,None
P.shape (1000, 1000)
q.shape (1000, 1)
G.shape (2000, 1000)
h.shape (2000, 1)
A.shape (1, 1000)
b.shape (1, 1)
     pcost       dcost       gap    pres   dres
 0: -8.7900e+02 -1.8790e+03  4e+03  1e+00  9e-17
 1: -7.3038e+02 -1.2264e+03  5e+02  1e-02  2e-16
 2: -8.6306e+02 -8.8351e+02  2e+01  6e-04  3e-16
 3: -8.7884e+02 -8.7905e+02  2e-01  6e-06  3e-16
 4: -8.7900e+02 -8.7900e+02  2e-03  6e-08  3e-16
 5: -8.7900e+02 -8.7900e+02  2e-05  6e-10  2e-16
Optimal solution found.
Accuracy: 0.879
x_controls=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 

ZeroDivisionError: float division by zero

In [38]:
X_svm, y_svm, x_control_svm = load_bank_data(load_data_size=data_count, drop_sensitive=False)
#X_svm, y_svm, x_control_svm = X, y, x_control
#print(y.shape)

C = C_1 # SVM regularization parameter
svc = svm.SVC(kernel='rbf', C=1).fit(X_svm, y_svm)

y_pred_svm = svc.predict(X_svm)
accuracy_svm = accuracy_score(y_svm, y_pred_svm)
print(y_pred_svm)
print ("Accuracy of SVM Classifier:", accuracy_svm)

Index(['job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'education_secondary',
       'education_tertiary', 'education_unknown', 'default_yes', 'housing_yes',
       'loan_yes', 'contact_telephone', 'contact_unknown', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown', 'y_yes', 'month-seasonal_q2',
       'month-seasonal_q3', 'month-seasonal_q4', 'balance', 'day', 'duration',
       'campaign', 'pdays', 'previous', 'age-range'],
      dtype='object')
0    879
1    121
Name: y_yes, dtype: int64
Shapes of X = (1000, 34), y=(1000,), x_sensitive=(1000,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 

/home/vipulpopat/Tools/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/vipulpopat/Tools/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
p_rule = compute_p_rule(x_control, y_pred_svm)
print(f'p_rule on SVM classification ={p_rule}')